In [6]:
from pytube import YouTube

import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
import math

In [7]:
def download_youtube_video(url, path):
    yt = YouTube(url)
    video = yt.streams.filter(file_extension='mp4').first()
    video.download(path)

In [3]:
download_youtube_video('https://www.youtube.com/watch?v=pcNiupgBh1M&ab_channel=HecticHockey', '/Users/seanfarquharson/Documents/vids_for_joint_detect')

In [21]:
# Load the MoveNet SinglePose Lightning model
model = hub.load('https://tfhub.dev/google/movenet/singlepose/lightning/4')
movenet = model.signatures['serving_default']

def process_frame(frame):
    # Resize and convert the image to a tensor
    frame = tf.image.resize_with_pad(frame, 192, 192)
    input_tensor = tf.cast(frame, dtype=tf.int32)  # Cast to int32 as required by the model

    # Run model inference
    output = movenet(tf.constant(input_tensor)[tf.newaxis, ...])
    keypoints = output['output_0'].numpy()

    return keypoints[0][0]  # Extract the first keypoint array

# Load your video
cap = cv2.VideoCapture('/Users/seanfarquharson/Documents/vids_for_joint_detect/lucas_swing.mov')

# Standard width and height
width, height = 1280, 720

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_swing_Lucas.avi', fourcc, 20.0, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to RGB (OpenCV uses BGR) and to a float32 tensor
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_tensor = tf.convert_to_tensor(frame_rgb, dtype=tf.float32)

    # Detect joints in the frame
    keypoints = process_frame(frame_tensor)

    # Draw keypoints
    for keypoint in keypoints:
        y, x, confidence = keypoint
        if confidence > 0.2:  # Adjust the threshold as needed
            cv2.circle(frame, (int(x * frame.shape[1]), int(y * frame.shape[0])), 5, (0, 255, 0), -1)

    out.write(frame)        
    
    # Display the frame
    cv2.imshow('MoveNet - Hockey Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

In [31]:
# Load the MoveNet SinglePose Lightning model
model = hub.load('https://tfhub.dev/google/movenet/singlepose/lightning/4')
movenet = model.signatures['serving_default']

def calculate_angle(a, b, c):
    # a, b and c are tuples containing the x, y coordinates of the joints
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

def process_frame(frame, original_width, original_height):
    # Resize for model input and convert to tensor
    input_frame = tf.image.resize(frame, (192, 192))
    input_tensor = tf.cast(input_frame, dtype=tf.int32)

    # Run model inference
    output = movenet(tf.constant(input_tensor)[tf.newaxis, ...])
    keypoints = output['output_0'].numpy()[0][0]

    # Scale keypoints back to original frame size
    scaled_keypoints = []
    for keypoint in keypoints:
        y, x, confidence = keypoint
        scaled_x = int(x * original_width)
        scaled_y = int(y * original_height)
        scaled_keypoints.append((scaled_x, scaled_y, confidence))

    return scaled_keypoints

cap = cv2.VideoCapture('/Users/seanfarquharson/Documents/vids_for_joint_detect/lucas_swing.mov')

# Get the original dimensions of the first frame
ret, test_frame = cap.read()
if not ret:
    print("Failed to read video")
    cap.release()
    exit()

original_height, original_width = test_frame.shape[:2]
cap.release()  # Reset the capture to start from the beginning

# Reinitialize the capture and VideoWriter with original dimensions
cap = cv2.VideoCapture('/Users/seanfarquharson/Documents/vids_for_joint_detect/lucas_swing.mov')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_swing_Lucas.avi', fourcc, 20.0, (original_width, original_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to RGB (OpenCV uses BGR) and to a float32 tensor
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_tensor = tf.convert_to_tensor(frame_rgb, dtype=tf.float32)

    # Detect and scale joints in the frame
    keypoints = process_frame(frame_tensor, original_width, original_height)

    # Draw keypoints and calculate angles
    for i, keypoint in enumerate(keypoints):
        x, y, confidence = keypoint
        if confidence > 0.2:
            cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)  # Draw green circle

            # Example: Calculate the angle between shoulder, elbow, and wrist
            if i in [5, 6, 7]:  # Indices for shoulder, elbow, and wrist
                if all(keypoints[j][2] > 0.2 for j in [5, 6, 7]):
                    angle = calculate_angle(keypoints[5][:2], keypoints[6][:2], keypoints[7][:2])
                    cv2.putText(frame, f"Angle: {int(angle)}", 
                                (x + 10, y + 10),  # Position the text
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    out.write(frame)  # Write the processed frame to the output video

    # Display the frame
    cv2.imshow('MoveNet - Golf Swing Analysis', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

In [22]:
import cv2
import numpy as np

width, height = 640, 480
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('test_output.avi', fourcc, 20.0, (width, height))

for _ in range(60):  # Write 60 frames
    frame = np.random.randint(0, 255, (height, width, 3), dtype=np.uint8)
    out.write(frame)

out.release()
